<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [ ]:
#импорт модулей
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, get_feature_names_out #make_column_transformer

In [ ]:
#загрузка данных
df = pd.read_csv('/datasets/Churn.csv', header = 0) 

Сохраним датасет в переменную df при помощи метода read_csv и выведем первые пять строк таблицы:

In [ ]:
#первые 5 строк 
df.head()

Признаки
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
-Gender — пол
-Age — возраст
-Tenure — сколько лет человек является клиентом банка
-Balance — баланс на счёте
-NumOfProducts — количество продуктов банка, используемых клиентом
-HasCrCard — наличие кредитной карты
-IsActiveMember — активность клиента
-EstimatedSalary — предполагаемая зарплата


-Целевой признак
-Exited — факт ухода клиента

In [ ]:
#Математическое описание значений по всей выборке
display(df.describe())

Возраст Age от 18 до 92, а баланс Balance от 0 до 250,000+ рублей. Из-за этого модель может посчитать, что баланс важнее возраста. 
Так как нам важны оба признака, применим нормализацию в следующей части исследования.

In [ ]:
#Математическое описание значений для неактивных клиентов
display(df[df['IsActiveMember'] == 0].describe())

Видим, что неактивны клиенты со средним возрастом в 37 лет(25 - 75 перцентиль состоит из возраста 32 - 43 года), состоявшиеся в работе клиенты (медиана зарплаты 100,000), половина из них имела лишь по одному банковскому продукту (50 перцентиль = 1), с кредитным рейтингом выше среднего (средний 650), и с медианным балансом в $1000.

В нашей выборке из 10,000 клиентов почти половина (4849) перестали быть активными. 

In [ ]:
#выгрузил пропуски, нашел долю пропусков от общего кол-ва значений
print(((df.isna().sum()/len(df)*100).round(2)).sort_values())

9% пропусков есть только в столбце Tenure: "сколько лет человек является клиентом банка". Так как значений не много, строки  пропущенными значениями:
- можно удалить эти строки 
- заменить пропущенное значение на определенное значение на значение из следующей строки, df['Tenure'] = df['Tenure'].fillna(method='bfill')
- заменить пропущенное значение на определенное значение на 0, этот способ и применим 
- заменить пропущенное значение  на медианное значение, расчитанное в зависимости от пола и возраста

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
print('Количество пропущенных значений для Tenure:', df['Tenure'].isna().sum())

In [ ]:
#Дублей в таблице при построчном сравнении нет
display(df.duplicated().sum())

In [ ]:
#Дублей по уникальным id клиентов нет
display(df['CustomerId'].duplicated().sum())

In [ ]:
#Информация о колонках и типах переменных
df.info()

Всего в таблице 10000 записей, при этом типы переменных в колонках соответствуют данным.

Удалим из таблицы столбец RowNumber, отражающий индекс строки в данных, так как он не содержит никакой полезной информации, при этом лишняя информация может негативно повлиять на адекватность и результативность модели.

Также на время подготовки данных к построению модели мы можем исключить из таблицы столбцы CustomerId, Surname, так как ни уникальный идентификатор клиента, ни фамилия не влияют на отношения банка и клиентов.

In [ ]:
#Удаляю не важные для исследования столбцы
df.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

Чтобы избежать мультиколлинеарность, посмотрим наколько сильно зависимы друг от друга числовые значения. Создадим список числовых значений, исключив столбцы с категориальными значениями.

In [ ]:
numeric = ['CreditScore','Age','Balance','NumOfProducts','EstimatedSalary','Tenure']

Расчитаем значение кореляции для стобцов, указанных выше.

In [ ]:
df.loc[:, numeric].corr()

Сильной корреляционной зависимости не обнаружено, максимальное значение 0.3, дополнительного удаления данных не потребуется.

В этой части проекта мы:
- проанализировал признаки
- удалили столбцы с избыточной для построения и обучения модели информацией ('RowNumber', 'CustomerId', 'Surname')
- нашли и заполнили пропуски в столбце 'Tenure'

## Исследование задачи

Деление на обучающую, валидационную и тестовую выборки

In [ ]:
# Данные подготовим методом OHE, что позволит нам использовать разные модели и не словить дамми ловушку
df_ml = pd.get_dummies(df, drop_first=True)
df_ml.head()

In [ ]:
target = df_ml['Exited']

features = df_ml.drop('Exited', axis=1)

In [ ]:
#разобьем на тестовую выборку (25%) для проверки итоговой модели и тренировочно-валидационную выборку для обучения и 
#поиска лучшей можели

features_valid_train, features_test, target_valid_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345, stratify=target
)
print(f"Количество строк в y_train_val по классам: {np.bincount(target_valid_train)}")
print(f"Количество строк в y_test по классам: {np.bincount(target_test)}")

#X_train_val, X_test, y_train_val, y_test = train_test_split(
#    features, target, test_size=0.25, random_state=12345)
#print(f"Количество строк в y_train_val по классам: {np.bincount(y_train_val)}")
#print(f"Количество строк в y_test по классам: {np.bincount(y_test)}")

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_valid_train, target_valid_train, test_size=0.25, 
    random_state=12345, stratify=target_valid_train)

print(f"Количество строк в y_train по классам: {np.bincount(target_train)}")
print(f"Количество строк в y_val по классам: {np.bincount(target_valid)}")

#features_train, features_valid, target_train, target_valid = train_test_split(
#    X_train_val, y_train_val, test_size=0.25, random_state=12345)

#print(f"Количество строк в y_train по классам: {np.bincount(target_train)}")
#print(f"Количество строк в y_val по классам: {np.bincount(target_valid)}")

In [ ]:
#Для масштабирования методом scaler зафиксируем численные признаки
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])

In [ ]:
#Масштабируем числ признаки обучающей выборки
features_train[numeric] = scaler.transform(features_train[numeric])
features_train.head()

In [ ]:
#Масштабируем численные признаки валидационной выборки 
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_valid.head()

In [ ]:
#Масштабируем численные признаки тестовой выборки 
features_test[numeric] = scaler.transform(features_test[numeric])
features_test.head()

In [ ]:
def all_models_f1accuracy(features_train, target_train, features_valid, target_valid):
    model_DTC = DecisionTreeClassifier(random_state=12345)
    DTC_fit = model_DTC.fit(features_train, target_train)
    DTC_predictions_valid = model_DTC.predict(features_valid)
    DTC_f1score= f1_score(target_valid, DTC_predictions_valid)
    
    model_RFC = RandomForestClassifier(max_depth=7, n_estimators=43, min_samples_leaf=3, random_state=12345)
    RFC_fit = model_RFC.fit(features_train, target_train)
    RFC_predictions_valid = model_RFC.predict(features_valid)
    RFC_f1score= f1_score(target_valid, RFC_predictions_valid)
    
    model_LgR = LogisticRegression(solver = 'liblinear')
    LgR_fit = model_LgR.fit(features_train, target_train)
    LgR_predictions_valid = model_LgR.predict(features_valid)
    LgR_f1score= f1_score(target_valid, LgR_predictions_valid)
    
    print("F1 score: ", "дерево решений", DTC_f1score, "случайный лес ", RFC_f1score, "логистческая регрессия", LgR_f1score,  sep='\n')

In [ ]:
all_models_f1accuracy(features_train, target_train, features_valid, target_valid)

In [ ]:
target_train.value_counts(normalize = 1)

In [ ]:
# Данные подготовим методом OHE, что позволит нам использовать разные модели и не словить дамми ловушку
df_ml = pd.get_dummies(df, drop_first=True)
df_ml.head()

In [ ]:
transformer = make_column_transformer(
    (OneHotEncoder(), ['Geography', 'Gender']),
    remainder='passthrough')
transformed = transformer.fit_transform(features_test_nc)
features_test = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
display(features_test.head())

In [ ]:
transformer = make_column_transformer(
    (OneHotEncoder(), ['Geography', 'Gender']),
    remainder='passthrough')
transformed = transformer.fit_transform(X_test)
features_test = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
display(features_test.head())

Onehot encoding

В таблице практически все столбцы, кроме Gender и Geography - количественные. Нам нужно построить модель, которая будет решать задачу классификации (уйдет клиент или нет). Применим технику прямого кодирования, чтобы преобразовать категориальные переменные в количественные.
Для избежания дамми-ловушки, удалим изначальный столбец drop_first=True

In [ ]:
#преобразую категориальные переменные в количественные
features_train = pd.get_dummies(features_train, drop_first=True, sparse = False)
features_valid = pd.get_dummies(features_valid, drop_first=True, sparse = False)
X_test = pd.get_dummies(X_test, drop_first=True, sparse = False)

#удаляю изначальные столбцы
#features_train.drop(["Gender", "Geography"], axis=1, inplace=True)
#features_valid.drop(["Gender", "Geography"], axis=1, inplace=True)
#X_test.drop(["Gender", "Geography"], axis=1, inplace=True)

#конкатенация 
#features_train = pd.concat([features_train, features_train_one_hot_gender], axis=1)
#features_valid = pd.concat([features_valid, features_valid_one_hot_gender], axis=1)
#X_test = pd.concat([X_test, X_test_one_hot_gender], axis=1)

#преобразую категориальные переменные в количественные
features_train_one_hot_gender = pd.get_dummies(features_train["Gender"], drop_first=True)
features_train_one_hot_geography = pd.get_dummies(features_train["Geography"], drop_first=True)

features_valid_one_hot_gender = pd.get_dummies(features_valid["Gender"], drop_first=True)
features_valid_one_hot_geography = pd.get_dummies(features_valid["Geography"], drop_first=True)

X_test_one_hot_gender = pd.get_dummies(X_test["Gender"], drop_first=True)
X_test_one_hot_geography = pd.get_dummies(X_test["Geography"], drop_first=True)

#удаляю изначальные столбцы
#features_train.drop(["Gender", "Geography"], axis=1, inplace=True)
#features_valid.drop(["Gender", "Geography"], axis=1, inplace=True)
#X_test.drop(["Gender", "Geography"], axis=1, inplace=True)

#конкатенация 
features_train = pd.concat([features_train, features_train_one_hot_gender, features_train_one_hot_geography], axis=1)
features_valid = pd.concat([features_valid, features_valid_one_hot_gender, features_valid_one_hot_geography], axis=1)
X_test = pd.concat([X_test, X_test_one_hot_gender, X_test_one_hot_geography], axis=1)

scaler

In [ ]:
#убрал предупреждение
pd.options.mode.chained_assignment = None

#привел поля ниже у диапазону 0-1, т.к. все признаки важны
scaler = StandardScaler()
scaler.fit(features_train)

features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
X_test = scaler.transform(X_test)

In [ ]:
features_train = pd.DataFrame(features_train)
features_valid = pd.DataFrame(features_valid)

target_train = pd.DataFrame(target_train)
target_valid = pd.DataFrame(target_valid)

X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)

In [ ]:
#Узнаем долю ушедших клиентов от всей выборки. Доля положительных объектов класса
sum(df["Exited"]/len(df))

In [ ]:
#Доля оставшихся клиентов от всей выборки.  Доля отрицательных объектов класса
1 - sum(df["Exited"]/len(df))

Доля положительного класса намного больше. Выборка не сбалансирована

Построим модели на несбалансированных данных 

In [ ]:
col = ['n_estimators', 'f1_score', 'auc_roc']
data = []

for est in range(1, 100):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train,target_train) 
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    data.append([est, f1, auc_roc])
table = pd.DataFrame(data = data, columns=col)
display(table[table['f1_score']==table['f1_score'].max()]) 
display(table[table['auc_roc']==table['auc_roc'].max()])

In [ ]:
model = LogisticRegression(random_state = 12345, solver='liblinear')
model.fit(features_train,target_train)
predicted_valid = model.predict(features_valid)

print("Cреднее гармоническое полноты и точности F1:", f1_score(target_valid, predicted_valid))

В этой части проекта мы:
- перевели столбцы с категориальными переменными в количественные ('Gender' и 'Geography')
- разделили данные на обучающую, валидационную и тестовую выборки
- обучили модель без учета дисбаланса 

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b>

Модели обучены корректно 👍
</div>

## Борьба с дисбалансом

У нас два пути решения этой проблемы:

- Upsampling
- Downsampling
- менять границу

Реализуем все 2 способа поочередно

upsample

In [ ]:
#увелечим положительный класс
#разделим обучающую выборку на отрицательные и положительные объекты     
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

#model = LogisticRegression(random_state = 12345, solver='liblinear')
#model.fit(features_upsampled, target_upsampled)
#predicted_valid = model.predict(features_valid)

#print("Cреднее гармоническое полноты и точности F1:", f1_score(target_valid, predicted_valid))

In [ ]:
features_upsampled.head()

In [ ]:
#источник https://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html 
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(features_upsampled)
from sklearn.utils import resample
features_upsampled, X_sparse, target_upsampled = resample(features_upsampled, X_sparse, target_upsampled, random_state=0)

features_upsampled.head()

In [ ]:
features_train.shape

In [ ]:
features_upsampled.shape
#было 5625 объектов, стало 8991 благодаря росту объектов положительного класса

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

#model = LogisticRegression(random_state = 12345, solver='liblinear')
#model.fit(features_downsampled, target_downsampled)
#predicted_valid = model.predict(features_valid)

#print("F1:", f1_score(target_valid, predicted_valid))

In [ ]:
features_downsampled.shape

было 5625, стало 1572 благодаря снижению объектов отрицательного класса. Далее буду работать с features_upsampled, т.к. там метрика f1 доходит почти до половины

## Тестирование модели

использую LogisticRegression, DecisionTreeClassifier, RandomForrestClassifier.
Для 2-х последних сначала найду лучшие параметры на валидациоонй выборке, при которых f1 максимальный. Потом для этих параметров построю графики для тестовой, валидационной и тестовой выборки

LogisticRegression

In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_upsampled, target_upsampled)#  (features_train, target_train)
predicted_valid = model.predict(features_valid)
print("для логитической регрессииF1:", f1_score(target_valid, predicted_valid))

Строю DecisionTreeClassifier 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Создаю списки для записи тестовых,  валидационных и тренировочных оценок 
f1_scores_test = []
f1_scores_val = []
f1_scores_train = []

#изначальное значение f1 и глубины = 0
best_f1 = 0
best_depth = 0

# итерация по разной глубине дерева
max_depths =  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for depth in max_depths:
    
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth , class_weight='balanced')
    #Обучаю на features_upsampled, данных после upsampled
    model.fit(features_upsampled, target_upsampled)#  (features_train, target_train)

    #предсказания для тестовой, валидационной и тестовой выборки
    predictions_test = model.predict(X_test)
    predictions_val = model.predict(features_valid)
    predictions_train = model.predict(features_train)
    
    #f1 для тестовой, валидационной и тестовой выборки
    f1_test = f1_score(y_test, predictions_test)
    f1_val = f1_score(target_valid, predictions_val)
    f1_train = f1_score(target_train, predictions_train)

    #заполняю списки для построения графика зависимости f1 от глубины дерева
    f1_scores_test.append(f1_test)
    f1_scores_val.append(f1_val)
    f1_scores_train.append(f1_train)
    
    if f1_val > best_f1:
        best_depth = depth
        f1_val = best_f1

# Plotting the graph
plt.figure(figsize=(20, 10))

plt.plot(max_depths, f1_scores_test , label='f1 Score на тестовой выборке')
plt.plot(max_depths, f1_scores_val , label='f1 Score на валидационной выборке')
plt.plot(max_depths, f1_scores_train , label='f1 Score на тренировочноый выборке')

plt.xlabel('Max Depth')
plt.ylabel('f1 Score')
plt.legend()
plt.show()

auc_roc_train = roc_auc_score(target_train, probabilities_one_train)
print('Тренировочная выборка:')
print('F1-мера:',max(f1_scores_train))
print('AUC-ROC:',auc_roc_train)
print()
auc_roc_val = roc_auc_score(target_valid, probabilities_one_val)
print('Валидационная выборка:')
print('F1-мера:', max(f1_scores_val))
print('AUC-ROC:',auc_roc_val)
print()
auc_roc_test = roc_auc_score(y_test, probabilities_one_test)
print('Тестовая выборка:')
print('F1-мера:',max(f1_scores_test))
print('AUC-ROC:',auc_roc_test)

Ищу лучшее количество деревьев для RandomForestClassifier

In [ ]:
col = ['n_estimators', 'f1_score', 'auc_roc']
data = []

#для количества деревьев от 1 до 99
for est in range(1, 100):
    model = RandomForestClassifier(random_state=12345, n_estimators=est,  class_weight='balanced') 
    model.fit(features_upsampled, target_upsampled)#  (features_train, target_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    data.append([est, f1, auc_roc])
table = pd.DataFrame(data = data, columns=col)
display(table[table['f1_score']==table['f1_score'].max()]) 
display(table[table['auc_roc']==table['auc_roc'].max()])

Высокое f1  достигается при количестве деревьев, равном 77 и 97
Используя 77 деревьев, найдем глубину для самого  высокого значения f1 и roc.

In [ ]:
# Создаю списки для записи тестовых,  валидационных и тренировочных оценок 
f1_scores_test = []
f1_scores_val = []
f1_scores_train = []


# итерация по разной глубине дерева
max_depths =  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for depth in max_depths:
    
    #изначальное значение f1 и глубины = 0
    best_f1 = 0
    best_depth = 0
    
    model = RandomForestClassifier(random_state=12345, n_estimators=77 , max_depth=depth,  class_weight='balanced')
    #Обучаю на features_upsampled, данных после upsampled
    model.fit(features_upsampled, target_upsampled)#  (features_train, target_train)

    #предсказания для тестовой, валидационной и тестовой выборки
    predictions_test = model.predict(X_test)
    predictions_val = model.predict(features_valid)
    predictions_train = model.predict(features_train)
    
    #f1 для тестовой, валидационной и тестовой выборки
    f1_test = f1_score(y_test, predictions_test)
    f1_val = f1_score(target_valid, predictions_val)
    f1_train = f1_score(target_train, predictions_train)

    #заполняю списки для построения графика зависимости f1 от глубины дерева
    f1_scores_test.append(f1_test)
    f1_scores_val.append(f1_val)
    f1_scores_train.append(f1_train)
   
    #запоминаю наибольшее значение f1 и глубины для него
    if f1_val > best_f1:
        best_depth = depth
        f1_val = best_f1
print(best_depth)  #выводит 10, что не верно
print(best_f1) #выводит 10, что не верно

# График зависимости f1 от глубины дерева
#размер
plt.figure(figsize=(20, 10))
#оси х,y, подпись графиков
plt.plot(max_depths, f1_scores_test , label='f1 Score на тестовой выборке')
plt.plot(max_depths, f1_scores_val , label='f1 Score на валидационной выборке')
plt.plot(max_depths, f1_scores_train , label='f1 Score на тренировочноый выборке')
#подписи осей и построение графика
plt.xlabel('Max Depth')
plt.ylabel('f1 Score')
plt.legend()
plt.show()

#график 'ROC-кривой'
fpr_valid, tpr_valid, thresholds = roc_curve(target_valid, probabilities_one_val)
fpr_train, tpr_train, thresholds = roc_curve(target_train, probabilities_one_train)
fpr_test, tpr_test, thresholds = roc_curve(y_test, probabilities_one_test)

plt.figure()
plt.plot(fpr_valid, tpr_valid)
plt.plot(fpr_train, tpr_train)
plt.plot(fpr_test, tpr_test)

plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show()

#печать f1, auc-roc
auc_roc_train = roc_auc_score(target_train, probabilities_one_train)
print('Тренировочная выборка:')
print('F1-мера:',f1_train)
print('AUC-ROC:',auc_roc_train)
print()
auc_roc_val = roc_auc_score(target_valid, probabilities_one_val)
print('Валидационная выборка:')
print('F1-мера:', f1_val)
print('AUC-ROC:',auc_roc_val)
print()
auc_roc_test = roc_auc_score(y_test, probabilities_one_test)
print('Тестовая выборка:')
print('F1-мера:',f1_test)
print('AUC-ROC:',auc_roc_test)


Теперь проверим на адекватность, выбранную нами модель. Для этого посмотрим, как справится с задачей фиктивный классификатор DummyClassifier. Он покажет вероятность успеха, даже если просто "угадать". Таким образом, чтобы признать модель адекватной, нужно, чтобы показатели метрик были выше, чем те, которые получатся у фиктивного классификатора. Так как от значения параметра strategy может поменяться результат, переберем их, и для сравнения будем учитывать лучший результат:

In [ ]:
data = []
#названия стратегий и метрик
names = ['stratified', 'most_frequent', 'prior', 'uniform']
col = ['name', 'f1_score_valid', 'auc_roc_valid', 'f1_score_test', 'auc_roc_test']
for name in names:
    #обучим на тренировочной выборке
    new_dummy_classifier = DummyClassifier(strategy=name)
    new_dummy_classifier.fit(features_train, target_train)
    #предскажем на валидационной
    predictions_valid = new_dummy_classifier.predict(features_valid)
    #расчет accuracy,f1, auc_roc 
    valid_accuracy = accuracy_score(target_valid, predictions)
    valid_f1_score = f1_score(target_valid, predictions_valid )
    auc_roc_valid = roc_auc_score(target_valid, predictions_valid)
     #предскажем на тестовой
    test_predictions = new_dummy_classifier.predict(X_test)
    #расчет accuracy,f1, auc_roc 
    test_accuracy = accuracy_score(y_test, test_predictions)
    test_f1_score = f1_score(y_test, test_predictions )
    auc_roc_test = roc_auc_score(y_test, test_predictions)
    #запишем все данные в список и выведем наибольший f1
    data.append([name, valid_f1_score, auc_roc_valid, test_f1_score, auc_roc_test])
table = pd.DataFrame(data = data, columns=col)
display(table[table['f1_score_test']==table['f1_score_test'].max()]) 
display(table[table['f1_score_valid']==table['f1_score_valid'].max()].head())

Вывод допишу на второй итерации. Надеюсь на подсказу про алгоритм

В качестве лучшей модели была выбрана модель случайного леса при значении гиперпараметра max_depth = 5, указании атрибута class_weight='balanced'.

На валидационной выборке модель следующие результаты:

F1-мера: 
AUC-ROC:
На тестовой выборке модель следующие результаты:

F1-мера: 0.60 (изменение на )
AUC-ROC: 0.836 (изменение на )
Проверку на адекватность модель выдержала. Лучшие (F1-мера:0.31, AUC-ROC: 0.52) показатели фиктивного классификатора значительно ниже.

Требование касательно значения F1-меры (нужно довести метрику до 0.59) выполнено - значение F1-меры нашей модели больше 0.6 на тестовой выборке.

# <font color='orange'>Общее впечатление</font>
* Этот проект выполнен очень хорошо
* Видно, что приложено много усилий
* Молодец, что структурируешь ноутбук, приятно проверять такие работы
* У тебя чистый и лаконичный код
* Мне было интересно читать твои промежуточные выводы
* Твой уровень подачи материала находится на высоком уровне
* Исправь, пожалуйста, мои замечания. Затем отправляй на повторную проверку
* Жду новую версию проекта 👋

# <font color='orange'>2. Общее впечатление</font>
* Спасибо за быстрое внесение правок
* Теперь проект выглядит лучше )
* Критических замечаний нет
* Молодец, отличная работа!
* Надеюсь, ревью было полезным
* Удачи в дальнейшем обучении 👋